In [28]:
import pandas as pd
import os
from bs4 import BeautifulSoup
import re
import nltk.data
nltk.download()
from nltk.corpus import stopwords
from gensim.models import word2vec
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [2]:
# Read data from files 
path_raw = os.getcwd().replace('notebooks', 'data/raw/')

train = pd.read_csv(path_raw + "labeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )

test = pd.read_csv(path_raw + "testData.tsv", header=0, delimiter="\t", quoting=3 )

unlabeled_train = pd.read_csv(path_raw + "unlabeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )

In [3]:
# Verify the number of reviews that were read (100,000 in total)
print("Read %d labeled train reviews, %d labeled test reviews, and %d unlabeled reviews\n" % (train["review"].size,  
test["review"].size, unlabeled_train["review"].size ))

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



In [4]:
def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [5]:
# for sentence splitting
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [24]:
raw_sentences = tokenizer.tokenize(train["review"][0].strip())

In [25]:
print(raw_sentences[0])
print(raw_sentences[1])
print(raw_sentences[2])

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again.
Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent.
Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released.


In [12]:
# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [13]:
sentences = []  # Initialize an empty list of sentences

print ("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print ("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set


/home/larissa/miniconda3/envs/kaggle/lib/python3.7/site-packages/bs4/__init__.py:302: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/home/larissa/miniconda3/envs/kaggle/lib/python3.7/site-packages/bs4/__init__.py:375: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Parsing sentences from unlabeled set


/home/larissa/miniconda3/envs/kaggle/lib/python3.7/site-packages/bs4/__init__.py:375: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/larissa/miniconda3/envs/kaggle/lib/python3.7/site-packages/bs4/__init__.py:375: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/larissa/miniconda3/envs/kaggle/lib/python3.7/site-packages/bs4/__init__.py:375: UserWarning: "http://www.msnbc.msn.com/id/4972055/site/newsweek/"" looks like a URL. Beautiful Soup is not an HTTP client. 

In [23]:
print("Number os setences")
print(len(sentences))
print("First setence")
print(sentences[0])
print("Second setence")
print(sentences[1])

Number os setences
795538
First setence
['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']
Second setence
['maybe', 'i', 'just', 'want', 'to', 'get', 'a', 'certain', 'insight', 'into', 'this', 'guy', 'who', 'i', 'thought', 'was', 'really', 'cool', 'in', 'the', 'eighties', 'just', 'to', 'maybe', 'make', 'up', 'my', 'mind', 'whether', 'he', 'is', 'guilty', 'or', 'innocent']


### Word2Vec

In [29]:
# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

print ("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)


2019-11-12 23:18:59,534 : INFO : collecting all words and their counts
2019-11-12 23:18:59,535 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-11-12 23:18:59,637 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2019-11-12 23:18:59,704 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types


Training model...


2019-11-12 23:18:59,781 : INFO : PROGRESS: at sentence #30000, processed 671314 words, keeping 30034 word types
2019-11-12 23:18:59,866 : INFO : PROGRESS: at sentence #40000, processed 897814 words, keeping 34348 word types
2019-11-12 23:18:59,933 : INFO : PROGRESS: at sentence #50000, processed 1116962 words, keeping 37761 word types
2019-11-12 23:18:59,993 : INFO : PROGRESS: at sentence #60000, processed 1338403 words, keeping 40723 word types
2019-11-12 23:19:00,066 : INFO : PROGRESS: at sentence #70000, processed 1561579 words, keeping 43333 word types
2019-11-12 23:19:00,139 : INFO : PROGRESS: at sentence #80000, processed 1780886 words, keeping 45714 word types
2019-11-12 23:19:00,213 : INFO : PROGRESS: at sentence #90000, processed 2004995 words, keeping 48135 word types
2019-11-12 23:19:00,286 : INFO : PROGRESS: at sentence #100000, processed 2226966 words, keeping 50207 word types
2019-11-12 23:19:00,354 : INFO : PROGRESS: at sentence #110000, processed 2446580 words, keeping 

2019-11-12 23:19:06,119 : INFO : PROGRESS: at sentence #750000, processed 16771405 words, keeping 120296 word types
2019-11-12 23:19:06,199 : INFO : PROGRESS: at sentence #760000, processed 16990809 words, keeping 120931 word types
2019-11-12 23:19:06,276 : INFO : PROGRESS: at sentence #770000, processed 17217946 words, keeping 121704 word types
2019-11-12 23:19:06,365 : INFO : PROGRESS: at sentence #780000, processed 17448092 words, keeping 122403 word types
2019-11-12 23:19:06,435 : INFO : PROGRESS: at sentence #790000, processed 17675168 words, keeping 123067 word types
2019-11-12 23:19:06,478 : INFO : collected 123505 word types from a corpus of 17798269 raw words and 795538 sentences
2019-11-12 23:19:06,479 : INFO : Loading a fresh vocabulary
2019-11-12 23:19:06,573 : INFO : effective_min_count=40 retains 16490 unique words (13% of original 123505, drops 107015)
2019-11-12 23:19:06,574 : INFO : effective_min_count=40 leaves 17239123 word corpus (96% of original 17798269, drops 559

2019-11-12 23:20:08,248 : INFO : EPOCH 2 - PROGRESS: at 52.89% examples, 329020 words/s, in_qsize 7, out_qsize 1
2019-11-12 23:20:09,248 : INFO : EPOCH 2 - PROGRESS: at 55.26% examples, 327995 words/s, in_qsize 7, out_qsize 0
2019-11-12 23:20:10,250 : INFO : EPOCH 2 - PROGRESS: at 57.84% examples, 328303 words/s, in_qsize 7, out_qsize 0
2019-11-12 23:20:11,265 : INFO : EPOCH 2 - PROGRESS: at 60.12% examples, 326602 words/s, in_qsize 7, out_qsize 0
2019-11-12 23:20:12,279 : INFO : EPOCH 2 - PROGRESS: at 61.97% examples, 322724 words/s, in_qsize 8, out_qsize 2
2019-11-12 23:20:13,314 : INFO : EPOCH 2 - PROGRESS: at 63.89% examples, 319142 words/s, in_qsize 8, out_qsize 0
2019-11-12 23:20:14,327 : INFO : EPOCH 2 - PROGRESS: at 66.28% examples, 318554 words/s, in_qsize 8, out_qsize 1
2019-11-12 23:20:15,328 : INFO : EPOCH 2 - PROGRESS: at 69.31% examples, 321023 words/s, in_qsize 8, out_qsize 1
2019-11-12 23:20:16,330 : INFO : EPOCH 2 - PROGRESS: at 72.66% examples, 324787 words/s, in_qsiz

2019-11-12 23:21:14,294 : INFO : EPOCH 4 - PROGRESS: at 14.22% examples, 353675 words/s, in_qsize 7, out_qsize 0
2019-11-12 23:21:15,295 : INFO : EPOCH 4 - PROGRESS: at 17.41% examples, 361259 words/s, in_qsize 6, out_qsize 1
2019-11-12 23:21:16,300 : INFO : EPOCH 4 - PROGRESS: at 20.78% examples, 370443 words/s, in_qsize 7, out_qsize 0
2019-11-12 23:21:17,329 : INFO : EPOCH 4 - PROGRESS: at 24.10% examples, 375494 words/s, in_qsize 7, out_qsize 0
2019-11-12 23:21:18,360 : INFO : EPOCH 4 - PROGRESS: at 26.86% examples, 371437 words/s, in_qsize 6, out_qsize 1
2019-11-12 23:21:19,374 : INFO : EPOCH 4 - PROGRESS: at 29.21% examples, 363885 words/s, in_qsize 7, out_qsize 0
2019-11-12 23:21:20,408 : INFO : EPOCH 4 - PROGRESS: at 31.62% examples, 357107 words/s, in_qsize 7, out_qsize 0
2019-11-12 23:21:21,413 : INFO : EPOCH 4 - PROGRESS: at 33.72% examples, 349356 words/s, in_qsize 8, out_qsize 0
2019-11-12 23:21:22,437 : INFO : EPOCH 4 - PROGRESS: at 35.61% examples, 340696 words/s, in_qsiz

2019-11-12 23:22:23,194 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-11-12 23:22:23,221 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-11-12 23:22:23,246 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-11-12 23:22:23,249 : INFO : EPOCH - 5 : training on 17798269 raw words (12749625 effective words) took 35.7s, 357261 effective words/s
2019-11-12 23:22:23,250 : INFO : training on a 88991345 raw words (63747160 effective words) took 191.8s, 332304 effective words/s


In [31]:
# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
path_raw = os.getcwd().replace('notebooks', 'models/')
model_name = path_raw + "300features_40minwords_10context"
model.save(model_name)

2019-11-12 23:27:15,160 : INFO : precomputing L2-norms of word weight vectors
2019-11-12 23:27:15,185 : INFO : saving Word2Vec object under /home/larissa/Documents/Git/Kaggle/kaggle-bag-of-words-popcorn/models/300features_40minwords_10context, separately None
2019-11-12 23:27:15,187 : INFO : not storing attribute vectors_norm
2019-11-12 23:27:15,190 : INFO : not storing attribute cum_table
2019-11-12 23:27:15,639 : INFO : saved /home/larissa/Documents/Git/Kaggle/kaggle-bag-of-words-popcorn/models/300features_40minwords_10context


In [35]:
#The "doesnt_match" function will try to deduce which word in a set is most dissimilar from the other
model.doesnt_match("france england germany berlin".split())

/home/larissa/miniconda3/envs/kaggle/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  


'berlin'

In [36]:
model.most_similar("man")

/home/larissa/miniconda3/envs/kaggle/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('woman', 0.6364747285842896),
 ('lady', 0.5911375284194946),
 ('lad', 0.5755161046981812),
 ('monk', 0.5695915222167969),
 ('soldier', 0.5397183895111084),
 ('farmer', 0.5141299962997437),
 ('businessman', 0.5132678747177124),
 ('men', 0.5119340419769287),
 ('guy', 0.5112597942352295),
 ('person', 0.5017234086990356)]

In [37]:
model.most_similar("awful")

/home/larissa/miniconda3/envs/kaggle/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('terrible', 0.7591007351875305),
 ('horrible', 0.7340126633644104),
 ('atrocious', 0.7267952561378479),
 ('abysmal', 0.7008300423622131),
 ('dreadful', 0.6972824335098267),
 ('horrid', 0.6731525659561157),
 ('horrendous', 0.6721129417419434),
 ('appalling', 0.665844202041626),
 ('lousy', 0.6342982053756714),
 ('amateurish', 0.6170752048492432)]

In [41]:
from gensim.models import Word2Vec
model = Word2Vec.load(path_raw + "300features_40minwords_10context")

2019-11-13 22:21:28,761 : INFO : loading Word2Vec object from /home/larissa/Documents/Git/Kaggle/kaggle-bag-of-words-popcorn/models/300features_40minwords_10context
2019-11-13 22:21:31,447 : INFO : loading wv recursively from /home/larissa/Documents/Git/Kaggle/kaggle-bag-of-words-popcorn/models/300features_40minwords_10context.wv.* with mmap=None
2019-11-13 22:21:31,448 : INFO : setting ignored attribute vectors_norm to None
2019-11-13 22:21:31,449 : INFO : loading vocabulary recursively from /home/larissa/Documents/Git/Kaggle/kaggle-bag-of-words-popcorn/models/300features_40minwords_10context.vocabulary.* with mmap=None
2019-11-13 22:21:31,450 : INFO : loading trainables recursively from /home/larissa/Documents/Git/Kaggle/kaggle-bag-of-words-popcorn/models/300features_40minwords_10context.trainables.* with mmap=None
2019-11-13 22:21:31,452 : INFO : setting ignored attribute cum_table to None
2019-11-13 22:21:31,453 : INFO : loaded /home/larissa/Documents/Git/Kaggle/kaggle-bag-of-words

In [43]:
type(model.cum_table)


/home/larissa/miniconda3/envs/kaggle/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `cum_table` (Attribute will be removed in 4.0.0, use self.vocabulary.cum_table instead).
  """Entry point for launching an IPython kernel.


numpy.ndarray

In [47]:
model["flower"]

/home/larissa/miniconda3/envs/kaggle/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


(300,)

### One challenge with the IMDB dataset is the variable-length reviews. We need to find a way to take individual word vectors and transform them into a feature set that is the same length for every review.

In [ ]:
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
       if counter%1000. == 0.:
           print "Review %d of %d" % (counter, len(reviews))
       # 
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[counter] = makeFeatureVec(review, model, \
           num_features)
       #
       # Increment the counter
       counter = counter + 1.
    return reviewFeatureVecs
